In [1]:
!pip install tensorflow

In [2]:
!pip install tflearn

     |████████████████████████████████| 112kB 5.8MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-cp37-none-any.whl size=127301 sha256=6de129dd849fabb4b9ae810864d2e6399f64e93a0c0bf07223b106cd342b7e79
  Stored in directory: /root/.cache/pip/wheels/31/d2/ed/fb9a0d301dd9586c11e9547120278e624227f22fd5f4baf744
Successfully built tflearn


In [3]:


import numpy
import tflearn
import tensorflow
import random

Instructions for updating:
non-resource variables are not supported in the long term


In [17]:
import pickle

In [4]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

In [24]:
from google.colab import files
uploaded = files.upload()

Saving new.json to new.json


In [25]:
import json
with open('new.json') as file:
    data = json.load(file)

In [26]:
print(data["intents"])

[{'tag': 'greeting', 'patterns': ['Hi', 'How are you', 'Hey', 'Is anyone there?', 'Hello', 'Good day', 'Whats up'], 'responses': ['Hello!', 'Hey, this is DigiBot', 'Good to see you again!', 'Hi there, how can I help?', 'DigiBot to assist you'], 'context_set': ''}, {'tag': 'goodbye', 'patterns': ['bye', 'See you later', 'Goodbye', 'I am Leaving', 'Have a Good day'], 'responses': ['Sad to see you go :(', 'Talk to you later', 'Hope to see you soon', 'Goodbye!'], 'context_set': ''}, {'tag': 'Owner', 'patterns': ['I want to sell a property', 'I need a new tenant', 'I want to change my tenant', 'I want a new buyer', 'Rent agreement', 'Price for my apartment'], 'responses': ["You're on right track", "I'll help you with that", 'Let me connect you to our executive'], 'context_set': ''}, {'tag': 'Buyer', 'patterns': ['I want a new apartment', 'I want a new flat', 'Connect me with seller', 'Connect me with owner', 'property rate'], 'responses': ["You're at a right place", 'Please let me help you'

In [27]:
words = []
labels = []
docs_x = []
docs_y = []

In [28]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [29]:
for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])
        
    if intent['tag'] not in labels:
        labels.append(intent['tag'])

In [30]:
words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

In [31]:
training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)

In [32]:
training = numpy.array(training)
output = numpy.array(output)

In [33]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

In [34]:
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

In [35]:
model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
model.save("model.tflearn")

Training Step: 3999  | total loss: 0.02340 | time: 0.022s
| Adam | epoch: 1000 | loss: 0.02340 - acc: 0.9997 -- iter: 24/30
Training Step: 4000  | total loss: 0.02297 | time: 0.026s
| Adam | epoch: 1000 | loss: 0.02297 - acc: 0.9997 -- iter: 30/30
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [36]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)

In [37]:
def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])
        results_index = numpy.argmax(results)
        tag = labels[results_index]

        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']

        print(random.choice(responses))

chat()

Start talking with the bot (type quit to stop)!
You: hey
Hi there, how can I help?
You: Who are you
Talk to you later
You: Hi
DigiBot to assist you
You: I want to buy a new house
You're on right track
You: want to sell a place
You're on right track
You: Show me the rent
Please let me help you
You: connect me with seller
I'll connect you with an executive
You: quit
